In [1]:
%pip install contractions
%pip install textblob
%pip install nltk
%pip install scikit-learn
%pip install transformers
%pip install emoji
%pip install spacy
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.3/624.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 94.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import pandas as pd
import re
from sklearn.feature_extraction import text
import string
import contractions
import emoji
import spacy
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Loading the data:

In [ ]:
data = pd.read_csv("filtered_data.csv")
data.head()

,post_id,subreddit,post_title,post_body,number_of_comments,readable_datetime,post_author,number_of_upvotes,query,text,comment_id,comment_body,comment_author
0,1002dom,technology,"ChatGPT Caused 'Code Red' at Google, Report Says",NaN,370,2023-01-01 00:03:33,slakmehl,792,ChatGPT,"ChatGPT Caused 'Code Red' at Google, Report Sa...",NaN,NaN,NaN
1,101o6zx,singularity,"how could someone use ChatGPT or other ""AI"" se...",I'm just looking to make some extra cash on th...,6,2023-01-03 01:25:17,theferalturtle,4,gpt OR GPT or Gpt,"how could someone use ChatGPT or other ""AI"" se...",NaN,NaN,NaN
2,101p00n,OpenAI,Chat gpt won’t verify my phone number,Hello guys when i try to log in to chat gpt it...,73,2023-01-03 01:56:45,T-boner970,3,coding with ChatGPT,Chat gpt won’t verify my phone number Hello gu...,NaN,NaN,NaN
3,102ci8x,OpenAI,Is it possible to use Whisper function for zoo...,"Hello, I am a recent Mathematics undergraduate...",11,2023-01-03 20:34:00,LoanOne2968,1,Whisper,Is it possible to use Whisper function for zoo...,NaN,NaN,NaN
4,102jcse,OpenAI,How could ChatGPT replace Google Search for ma...,NaN,9,2023-01-04 01:03:24,keyhell,1,ChatGPT search,How could ChatGPT replace Google Search for ma...,NaN,NaN,NaN


Converting texts to lowercase:

In [5]:
def lower(text):
  return text.lower()

data["Cleaned Text"] = data["text"].apply(lower)
data["Cleaned Text"].head()

,Cleaned Text
0,"chatgpt caused 'code red' at google, report sa..."
1,"how could someone use chatgpt or other ""ai"" se..."
2,chat gpt won’t verify my phone number hello gu...
3,is it possible to use whisper function for zoo...
4,how could chatgpt replace google search for ma...


Removing r/, usernames, new line indicators, and links from texts:

In [6]:
def remove_links(text):
  return re.sub(r'http[s]?://\S+|www\.\S+', '', text)

def remove_user_mentions(text):
    return re.sub(r'u/\S+', '', text)

data["Cleaned Text"] = data["Cleaned Text"].str.rstrip(" none")
data["Cleaned Text"] = data["Cleaned Text"].str.replace('r/', '', regex=False)
data["Cleaned Text"] = data["Cleaned Text"].str.replace("\n\n", ' ', regex=False)
data["Cleaned Text"] = data["Cleaned Text"].apply(remove_links)
data["Cleaned Text"] = data["Cleaned Text"].apply(remove_user_mentions)

data["Cleaned Text"].head()

,Cleaned Text
0,"chatgpt caused 'code red' at google, report says"
1,"how could someone use chatgpt or other ""ai"" se..."
2,chat gpt won’t verify my phone number hello gu...
3,is it possible to use whisper function for zoo...
4,how could chatgpt replace google search for ma...


Expanding contractions:

In [7]:
def expand_contractions(text):
    return contractions.fix(text)

data["Cleaned Text"] = data["Cleaned Text"].apply(expand_contractions)
data["Cleaned Text"].head()

,Cleaned Text
0,"chatgpt caused 'code red' at google, report says"
1,"how could someone use chatgpt or other ""ai"" se..."
2,chat gpt will not verify my phone number hello...
3,is it possible to use whisper function for zoo...
4,how could chatgpt replace google search for ma...


Converting emojis to their descriptive names:

In [8]:
def convert_emojis(text):
   return emoji.demojize(text)

data["Cleaned Text"] = data["Cleaned Text"].apply(convert_emojis)
data["Cleaned Text"].head()

,Cleaned Text
0,"chatgpt caused 'code red' at google, report says"
1,"how could someone use chatgpt or other ""ai"" se..."
2,chat gpt will not verify my phone number hello...
3,is it possible to use whisper function for zoo...
4,how could chatgpt replace google search for ma...


Removing non-ASCII characters:

In [9]:
def remove_non_ascii(text):
  return text.encode("ascii", "ignore").decode()

data["Cleaned Text"] = data["Cleaned Text"].apply(remove_non_ascii)
data["Cleaned Text"].head()

,Cleaned Text
0,"chatgpt caused 'code red' at google, report says"
1,"how could someone use chatgpt or other ""ai"" se..."
2,chat gpt will not verify my phone number hello...
3,is it possible to use whisper function for zoo...
4,how could chatgpt replace google search for ma...


Removing punctuations:

In [10]:
def remove_punctuations(text):
    text = re.sub(r'[-]', ' ', text)
    text = re.sub(r'(\S)[' + re.escape(string.punctuation) + r'](\S)', r'\1 \2', text)
    return text

data["Cleaned Text"] = data["Cleaned Text"].apply(remove_punctuations)

pattern_punctuations = r'[' + string.punctuation + r']'

data["Cleaned Text"] = data["Cleaned Text"].str.replace(pattern_punctuations, '', regex=True)

data["Cleaned Text"].head()

,Cleaned Text
0,chatgpt caused code red at google report says
1,how could someone use chatgpt or other ai serv...
2,chat gpt will not verify my phone number hello...
3,is it possible to use whisper function for zoo...
4,how could chatgpt replace google search for ma...


Removing numbers:

In [11]:
def remove_numbers(text):
    return re.sub(r'[0-9]+', '', text)

data["Cleaned Text"] = data["Cleaned Text"].apply(remove_numbers)
data['Cleaned Text'].head()

,Cleaned Text
0,chatgpt caused code red at google report says
1,how could someone use chatgpt or other ai serv...
2,chat gpt will not verify my phone number hello...
3,is it possible to use whisper function for zoo...
4,how could chatgpt replace google search for ma...


### TF-IDF Vectorizer

In [12]:
nlp = spacy.load("en_core_web_sm")

def custom_tokenizer(text):
    doc = nlp(text)
    tokens = []
    for ent in doc.ents:
        tokens.append(ent.text)

    non_entity_tokens = [token.lemma_.lower() for token in doc if not token.ent_type_ and not token.is_punct and not token.is_space]
    tokens.extend(non_entity_tokens)
    return tokens

vectorizer = TfidfVectorizer(
    tokenizer=custom_tokenizer,
    use_idf=False,
    stop_words="english",
    min_df=5
)

doc_vectors = vectorizer.fit_transform(data["Cleaned Text"]).astype("float32")
feature_names = vectorizer.get_feature_names_out()
tfidf_data = pd.DataFrame.sparse.from_spmatrix(doc_vectors, columns=feature_names)
print(tfidf_data.head())

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['far', 'make', 'whereaft'] not in stop_words.
  warnings.warn(


   \  \    a  hour  a  minute  a  month  a  year  a  year old  \
0  0    0        0          0         0        0            0   
1  0    0        0          0         0        0            0   
2  0    0        0          0         0        0            0   
3  0    0        0          0         0        0            0   
4  0    0        0          0         0        0            0   

   a billion dollars  a couple days  a couple months  ...  zara  zero  \
0                  0              0                0  ...     0     0   
1                  0              0                0  ...     0     0   
2                  0              0                0  ...     0     0   
3                  0              0                0  ...     0     0   
4                  0              0                0  ...     0     0   

   zerogpt  zip  zombie  zone      zoom  zooming  zuck  zuckerberg  
0        0    0       0     0         0        0     0           0  
1        0    0       0     0   

In [13]:
query = ["chatgpt", "performance", "user experiences", "applications", "limitations", "ethical concerns", "comparisons", "ai model", "openai", "deepseek", "claude",
         "chatbot", "gpt", "language model", "ai assistant", "nlp", "machine learning", "deep learning", "ai-generated content", "ai reasoning", "automated responses",
         "performance", "pricing", "cost", "privacy", "quality", "rate limit", "review", "security", "sora", "speed", "subscription", "chatgpt plus", "benchmarks",
         "quality", "latency", "llama"]

idf_vectorizer = TfidfVectorizer(
    tokenizer=custom_tokenizer,
    use_idf=True,
    # turn off sublinear_tf to get the same results as the previous implementation
    sublinear_tf=False,
    stop_words="english",
    # using the vocabulary from the matrix of the document because min_df=5 cannot be used here (it will ignore words appearing less than 5 times in the query too)
    vocabulary=vectorizer.vocabulary_
)

query_vector = idf_vectorizer.fit_transform(query)

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['far', 'make', 'whereaft'] not in stop_words.
  warnings.warn(


In [ ]:
similarity_scores = cosine_similarity(query_vector, doc_vectors)[0]

data["similarity"] = similarity_scores

sorted_data = data.sort_values(by="similarity", ascending=False)

sorted_data.to_csv("similarity_scores.csv", index=False)

print("Data saved to 'similarity_scores.csv'")

Data saved to 'similarity_scores.csv'


In [15]:
sorted_data.head()

,post_id,subreddit,post_title,post_body,number_of_comments,readable_datetime,post_author,number_of_upvotes,query,text,comment_id,comment_body,comment_author,Cleaned Text,similarity
5671,1icmp32,ChatGPT,Not much better than chatgpt,NaN,3,2025-01-29 09:26:17,IotaNine,1,NaN,Not much better than chatgpt None,NaN,NaN,NaN,not much better than chatgpt,1.0
41680,1hmv4v8,OpenAI,CHAT GPT IS DOWN.,NaN,380,2024-12-26 22:55:29,Crystaleana,25,availability OR downtime OR outage,https://preview.redd.it/hn2eap1u299e1.jpeg?wid...,m3x9pj6,https://preview.redd.it/hn2eap1u299e1.jpeg?wid...,SnooComics7329,not only chatgpt,1.0
35284,1ggixzy,ChatGPT,"AMA with OpenAI’s Sam Altman, Kevin Weil, Srin...",Consider this AMA our Reddit launch. Ask us a...,4713,2024-10-31 20:40:38,OpenAI,120,NaN,does chatgpt have a mascot?,luq8n03,does chatgpt have a mascot?,Positive-Fall-787,does chatgpt have a mascot,1.0
5231,1i81o2x,ChatGPT,chatgpt down again :(,NaN,4,2025-01-23 15:54:00,ayansengupta17,1,o3,chatgpt down again :( None,NaN,NaN,NaN,chatgpt down again,1.0
42323,1hrng45,ChatGPT,"Yep, I finally got a cat",NaN,17,2025-01-02 10:21:38,General-Finger1159,1,o3,That’s why they call it ChatGPT,m4z0taz,That’s why they call it ChatGPT,jrdnmdhl,that is why they call it chatgpt,1.0
